# Exporting CSV data from the server

This process is slightly cumbersome because of Unix permissions.  Remember - nine times out of ten, on Unix, it's probably a permissions problem.

In this case the user '`postgres`' which runs the PostgreSQL server doesn't have write permissions to your home directory `/home/jovyan/work`.  To work around it, we write to a shared space, `/tmp`, from PostgreSQL, then copy the file to your own directory.

### Standard db setup 

In [3]:
!echo 'redspot' | sudo -S service postgresql restart

[sudo] password for jovyan: Restarting PostgreSQL 9.5 database server: main.


In [4]:
%load_ext sql

In [5]:
!createdb -U dbuser test

In [6]:
%sql postgresql://dbuser@localhost:5432/test

'Connected: dbuser@test'

### A little database example

In [11]:
%%sql
DROP TABLE IF EXISTS foo;
CREATE TABLE foo (
    id SERIAL,
    s TEXT
);

Done.
Done.


[]

In [18]:
%%sql
INSERT INTO foo (s) VALUES 
('hi'),
('bye'),
('yo')
;

3 rows affected.


[]

In [19]:
%%sql
SELECT * FROM foo;

3 rows affected.


id,s
1,hi
2,bye
3,yo


### Exporting to CSV

Now that you see we have a real table with real data, we export using the same `COPY` command we use for import.  The main differences are:

 * `COPY ... TO` instead of `COPY ... FROM`
 * You may specify an arbitrarily complex query, using multiple tables, etc.
 * Note the `/tmp/` location of the output file; this is our shared space.
 
Read all the details about pgsql's non-standard-SQL `COPY` function at https://www.postgresql.org/docs/9.5/static/sql-copy.html.

In [27]:
%%sql
COPY
(SELECT * FROM foo ORDER BY s)
TO '/tmp/testout.csv'
WITH
CSV
HEADER 
DELIMITER ','
QUOTE '"';

3 rows affected.


[]

We can see that the file correctly exported.

In [29]:
!cat /tmp/testout.csv

id,s
2,bye
1,hi
3,yo


In [33]:
!csvlook /tmp/testout.csv

|-----+------|
|  id | s    |
|-----+------|
|  2  | bye  |
|  1  | hi   |
|  3  | yo   |
|-----+------|


Now move the file to a space you can reach from Jupyter:

In [32]:
!cp /tmp/testout.csv /home/jovyan/work/testout.csv

At this point, you should be able to see the file added on Jupyter's file listing screen.  

From there, you can use the file like any other.


### Downloading the file to your local machine

In Jupyter's file list screen, click on the file to open it, then when it opens up in the text editor, use File -> Download to save it to your local machine.